# Tests for hapCON designed to run on Leipzig MPI EVA cluster

In [1]:
### Some Code to set right paths (checks whether on Chicago cluster)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os as os
import sys as sys
import multiprocessing as mp
import sys
import socket as socket

socket_name = socket.gethostname()
print(socket_name)

if socket_name.startswith("bionc") or socket_name.startswith("hpc"):
     path = "/mnt/archgen/users/hringbauer/git/hapROH/"   # The Path on Harald's machine
else: 
     raise RuntimeWarning("Not compatible machine. Check!!")
    
os.chdir(path)  # Set the right Path (in line with Atom default)

print(os.getcwd()) # Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns
print(f"CPU Count: {mp.cpu_count()}")

############################################
### If wanting to use local version and not  pip installed version
### Comment out to test installed version. Comment in to test development version:
### 

#sys.path.insert(0,"./package/")  # hack to get local UoC package first in path
#sys.path.insert(0, "/mnt/archgen/users/yilei/tools/hapROH/package")  ### Yilei's Version at MPI EVA
#sys.path.insert(0, "/mnt/archgen/users/hringbauer/git/hapROH/package")  ### Leipzig Version Harald

#from hapsburg.PackagesSupport.hapsburg_run import hapsb_ind
from hapsburg.PackagesSupport.hapsburg_run import hapCon_chrom_BFGS

hpc030
/mnt/archgen/users/hringbauer/git/hapROH
CPU Count: 128


In [2]:
def test_passed(value, acceptable=[]):
    """Check wether test passed.
    If value in acceptable [List length 2],
    return 0"""
    
    passed = (value > acceptable[0] and value < acceptable[1])
    
    exit = 1
    if passed:
        print(f"Test sussessful!")
        exit = 0
    else:
        print("ALARM ALARM ALARM")       
    return exit

## 1)a) hapCon with 1240k panel on a Sardinian, using Python function
- 35s on Leipzig cluster

In [3]:
%%time

### Relevant Paths
path2bam="/mnt/archgen/users/yilei/tools/hapROH/Notebooks/Vignettes/Data/SUA001.bam"
outpath = "/mnt/archgen/users/hringbauer/area51/tmp/SUA001.mpileup"
path2bed1240k="/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/1240kChrX.bed"
path2ref1240k="/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/chrX.hdf5"
path2meta="/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/meta_df_all.csv"


!samtools index $path2bam
!samtools mpileup --positions $path2bed1240k -r X -q 30 -Q 30 -o $outpath $path2bam

### Run the actual conatmination estimate
conmle = hapCon_chrom_BFGS(iid="SUA001", mpileup=outpath,
    h5_path1000g = path2ref1240k, meta_path_ref = path2meta)

test_passed(conmle[0], acceptable=[0.1,0.11])

[mpileup] 1 samples in 1 input files
exclude 1033 sites outside the specified region
exclude 0 non-SNP sites
time taken to read mpileup: 0.15 seconds
number of major reads at flanking sites: 10636
number of minor reads at flanking sites: 16
number of major reads at focal sites: 1291
number of minor reads at focal sites: 27
err rate at flanking sites: 0.001502
err rate at focal sites: 0.020486
saving sample as SUA001 in /mnt/archgen/users/hringbauer/area51/tmp/SUA001.hdf5
time taken to save hdf5: 0.06 seconds
estimated genotyping error by flanking sites: 0.001502
number of sites covered by at least one read: 3999, fraction covered: 0.085
hdf5 file saved to /mnt/archgen/users/hringbauer/area51/tmp/SUA001.hdf5
finished reading mpileup file, takes 0.359.
number of sites covered by at least one read: 3999
hdf5 file saved to /mnt/archgen/users/hringbauer/area51/tmp/SUA001.hdf5
Using Pre-Processing Model fm: HDF5
Loading H5
estimated contamination rate: 0.101227(0.075982 - 0.126472)
Test suss

0

### 1)b) Test running directly running from bam File, using Python function
- 1min11s on Leipzig cluster

In [4]:
%%time

# test running directly from bam file
path2bam="/mnt/archgen/users/yilei/tools/hapROH/Notebooks/Vignettes/Data/SUA001.bam"
outpath = "/mnt/archgen/users/hringbauer/area51/tmp/SUA001.mpileup"
#path2bed1240k="/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/1240kChrX.bed"
path2ref1240k="/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/chrX.hdf5"
path2meta="/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/meta_df_all.csv"

conmle = hapCon_chrom_BFGS(iid="SUA001", bam=path2bam,
    h5_path1000g = path2ref1240k, meta_path_ref = path2meta)

test_passed(conmle[0], acceptable=[0.1,0.11])

exclude 1033 sites outside the specified region
exclude 0 non-SNP sites
total number of mapped reads: 14755
number of major reads at flanking sites: 10521
number of minor reads at flanking sites: 15
number of major reads at focal sites: 1291
number of minor reads at focal sites: 27
err rate at flanking sites: 0.001424
err rate at focal sites: 0.020486
saving sample as SUA001 in /mnt/archgen/users/yilei/tools/hapROH/Notebooks/Vignettes/Data/SUA001.hdf5
estimated genotyping error by flanking sites: 0.001424
number of sites covered by at least one read: 3999, fraction covered: 0.085
hdf5 file saved to /mnt/archgen/users/yilei/tools/hapROH/Notebooks/Vignettes/Data/SUA001.hdf5
finished reading bam file, takes 54.748.
number of sites covered by at least one read: 3999
hdf5 file saved to /mnt/archgen/users/yilei/tools/hapROH/Notebooks/Vignettes/Data/SUA001.hdf5
Using Pre-Processing Model fm: HDF5
Loading H5
estimated contamination rate: 0.101304(0.076059 - 0.126550)
Test sussessful!
CPU times

0

## 2)a) Test 1000G panel and WGS XiongNu DA43 sample

In [5]:
%%time 

path2bam="/mnt/archgen/users/yilei/tools/hapROH/Notebooks/Vignettes/Data/DA43.bam"
path2bed1kg="/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/maf5FilterChrX.bed"
path2ref1kg="/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/maf5_filter_chrX.hdf5"
outpath = "/mnt/archgen/users/hringbauer/area51/tmp/DA43.mpileup"

!samtools index $path2bam
!samtools mpileup --positions $path2bed1kg -r X -q 30 -Q 30 -o $outpath $path2bam


conmle = hapCon_chrom_BFGS(iid="DA43", mpileup=outpath,
    h5_path1000g = path2ref1240k, meta_path_ref = path2meta)

test_passed(conmle[0], acceptable=[0.02,0.025])

[mpileup] 1 samples in 1 input files
exclude 1033 sites outside the specified region
exclude 0 non-SNP sites
time taken to read mpileup: 0.59 seconds
number of major reads at flanking sites: 18220
number of minor reads at flanking sites: 184
number of major reads at focal sites: 390
number of minor reads at focal sites: 13
err rate at flanking sites: 0.009998
err rate at focal sites: 0.032258
saving sample as DA43 in /mnt/archgen/users/hringbauer/area51/tmp/DA43.hdf5
time taken to save hdf5: 0.06 seconds
estimated genotyping error by flanking sites: 0.009998
number of sites covered by at least one read: 3595, fraction covered: 0.077
hdf5 file saved to /mnt/archgen/users/hringbauer/area51/tmp/DA43.hdf5
finished reading mpileup file, takes 0.933.
number of sites covered by at least one read: 3595
hdf5 file saved to /mnt/archgen/users/hringbauer/area51/tmp/DA43.hdf5
Using Pre-Processing Model fm: HDF5
Loading H5
estimated contamination rate: 0.022870(0.008922 - 0.036819)
Test sussessful!


0

### 2)b) Run XiongNu directly from .bam file

In [6]:
%%time

path2bam="/mnt/archgen/users/yilei/tools/hapROH/Notebooks/Vignettes/Data/DA43.bam"

conmle  = hapCon_chrom_BFGS(iid="DA43", bam=path2bam,
    h5_path1000g = path2ref1240k, meta_path_ref = path2meta)
test_passed(conmle[0], acceptable=[0.02,0.025])

exclude 1033 sites outside the specified region
exclude 0 non-SNP sites
total number of mapped reads: 281907
number of major reads at flanking sites: 4776
number of minor reads at flanking sites: 41
number of major reads at focal sites: 569
number of minor reads at focal sites: 16
err rate at flanking sites: 0.008512
err rate at focal sites: 0.027350
saving sample as DA43 in /mnt/archgen/users/yilei/tools/hapROH/Notebooks/Vignettes/Data/DA43.hdf5
estimated genotyping error by flanking sites: 0.008512
number of sites covered by at least one read: 5074, fraction covered: 0.108
hdf5 file saved to /mnt/archgen/users/yilei/tools/hapROH/Notebooks/Vignettes/Data/DA43.hdf5
finished reading bam file, takes 49.833.
number of sites covered by at least one read: 5074
hdf5 file saved to /mnt/archgen/users/yilei/tools/hapROH/Notebooks/Vignettes/Data/DA43.hdf5
Using Pre-Processing Model fm: HDF5
Loading H5
estimated contamination rate: 0.021418(0.008458 - 0.034378)
Test sussessful!
CPU times: user 33

0